In [3]:
import requests
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm
from IPython.display import clear_output

In [4]:
class LiteratureParser():
    
    def __init__(self):
        self.start_url = 'https://www.punjabi-kavita.com/'
        self.links = [self.start_url]
        self.all_links = self.links
        self.pairs = set()
        
    def get_html_page(self, url):
        try:
            response = requests.get(url)
            return response.text if response.ok else None
        except:
            return None
    
    def cut_link(self, s):
        ind = s.rfind('#')

        return s if ind == -1 else s[:ind]

    def find_all_links(self, soup, start_page=False):
        sections = soup.find_all('div', {'class': 'section'})
        sections = sections[:-1] if start_page else sections
        links = []
        for section in sections:
            tags_li = section.find_all('li')
            for tag_li in tags_li:
                try:
                    link = tag_li.find('a')['href']
                    link = self.cut_link(link)
                    full_link = self.start_url + link
                    
                    if full_link.find('.php') != -1 and links.count(full_link) == 0 and self.links.count(full_link) == 0:
                        if not full_link in self.all_links:
                            links.append(full_link)
                            self.all_links.append(full_link)
                    else:
                        pass
                except:
                    continue
        return links
    
    def get_texts(self, soup):
        titles = [tag.text for tag in soup.find_all('h3')]
        texts = [tag.text for tag in soup.find_all('p')]
        pairs = list(map(lambda x: '<<<>>>'.join(x), list(zip(titles, texts))))
        return pairs
    
    def save(self):
        with open('pa_literature2.txt', 'w') as f:
            for pair in self.pairs:
                f.write(pair)
                f.write('^^^^^')
        
    def parse_links(self, start_page=True):

        texts_counter = 0
        
        while True:
        
            cur_url = self.links[-1]
            self.links = self.links[:-1]
            html = self.get_html_page(cur_url)

            if html == -1: break    
            if not html: continue
                
            soup = BeautifulSoup(html)
            new_links = self.find_all_links(soup, start_page=start_page)
            
            self.links += new_links
            self.pairs = self.pairs.union(self.get_texts(soup))
            texts_counter += len(self.pairs)
            
            clear_output(wait=True)
            print(cur_url)
            print(len(new_links), len(self.links), texts_counter)
            
            if texts_counter >= 100:
                self.save()
                texts_counter = 0
            
            start_page=False        
        
            if not self.links: break
        self.save()

In [5]:
lit_parser = LiteratureParser()
lit_parser.parse_links()

https://www.punjabi-kavita.com/PoemsForChildren.php
0 0 28884
